# CHAPTER 2
# Working with Data via SQLAlchemy Core

We’ll look at how to:
* insert
* retrieve
* and delete data

and follow that with
learning how to:
* sort
* group
* and use relationships in our data. 

# Inserting Data

`<table.name>.insert().values()`

In [ ]:
ins = cookies.insert().values(
            cookie_name="chocolate chip",
            cookie_recipe_url="http://some.aweso.me/cookie/recipe.html",
            cookie_sku="CC01",
            quantity="12",
            unit_cost="0.50"
        )
print(str(ins))

 print(str(ins)) shows us the actual SQL statement

 ```
INSERT INTO cookies
(cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost)
VALUES
(:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)

 ```

values have been replaced with :column_name 
is how SQLAlchemy represents parameters displayed via the str() function.

Parameters are used to help ensure that our data has been properly escaped, which
mitigates security issues such as SQL injection attacks

 The compile() method on the ins object returns a SQLCom
piler object that gives us access to the actual parameters that will be sent



In [ ]:
ins.compile().params

we can use the execute() method on our connec‐
tion (the engine we created) to send the statement to the database,

In [ ]:
result = engine.execute(ins)


get the ID of the record we just inserted 

In [ ]:
result.inserted_primary_key


The ‘rowcount’ reports the number of rows matched by the WHERE criterion of an UPDATE or DELETE statement.

In [ ]:
result.rowcount

insert
 is also avail‐
able as a top-level function `insert(<table.name>).values()`

In [ ]:
from sqlalchemy import insert

ins = insert(cookies).values(
            cookie_name="chocolate chip",
            cookie_recipe_url="http://some.aweso.me/cookie/recipe.html",
            cookie_sku="CC01",
            quantity="12",
            unit_cost="0.50"
        )

It
is also possible to provide the values as keyword arguments to the execute method
after our statement. 

In [ ]:
ins = cookies.insert()
result = engine.execute(
            ins,
            cookie_name='dark chocolate chip',
            cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
            cookie_sku='CC02',
            unit_cost='0.75'
            )
result.inserted_primary_key

We can insert multiple records at once by using a list of dictionaries with
data we are going to submit. 

In [ ]:
inventory_list = [
        {
            'cookie_name': 'peanut butter',
            'cookie_recipe_url': 'http://some.aweso.me/cookie/peanut.html',
            'cookie_sku': 'PB01',
            'quantity': '24',
            'unit_cost': '0.25'
        },
        {
            'cookie_name': 'oatmeal raisin',
            'cookie_recipe_url': 'http://some.okay.me/cookie/raisin.html',
            'cookie_sku': 'EWW01',
            'quantity': '100',
            'unit_cost': '1.00'
        }
    ]
result = engine.execute(ins, inventory_list)

# Querying Data

start by using the select function

standard SQL SELECT statement. 

In [ ]:
from sqlalchemy.sql import select
s = select([cookies])
rp = engine.execute(s)
results = rp.fetchall()

use `str(s)` to look at the SQL statement the database will see.
`print(s)`


```
SELECT  cookies.cookie_id, 
        cookies.cookie_name, 
        cookies.cookie_recipe_url, 
        cookies.cookie_sku, 
        cookies.quantity, 
        cookies.unit_cost 
FROM cookies
```

It is also possible to use:

In [ ]:
from sqlalchemy.sql import select
s = cookies.select()
rp = engine.execute(s)
results = rp.fetchall()

# ResultProxy

main goal is to
make it easier to use and manipulate the results of a statement.


handling query results easier by allowing access using an index, name, or Column
object: 

In [ ]:
first_row = results[0]
first_row[1]
first_row.cookie_name
first_row[cookies.c.cookie_name]

Een overzicht van alle kolommen:

`print(cookies.c)`

We can also leverage the ResultProxy as an
iterable, and perform an action on each record returned without creating another
variable to hold the results. 

In [ ]:
rp = engine.execute(s)
for record in rp:
    print(record.cookie_name)

all the result variables in “Inserting Data” were actually ResultProxys.

rowcount() and inserted_primary_key() are ways to get information from a ResultProxy.

 following methods as well:

 
`first()`
Returns the first record if there is one 
AND 
closes the connection.


 `fetchone()`
 Returns one row 
 AND 
 leaves the cursor open 


 `scalar()`
 Fetch the first column of the first row, 
 AND
 close the result set.
Returns None if there are no rows to fetch.


`keys()`
method to get a list of the column names.
AND
closes the connection.


![](images/sqlalchemy_guide_for_good_production.png)

# ---------------------- HERE---------------------------

Every time we queried the database in the preceding examples, all the columns were
returned for every record. 

# Controlling the Columns in the Query

pass the columns we
want into the select() method

In [ ]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
rp = engine.execute(s)
print(rp.keys())
result = rp.first()

# Ordering

we can chain an order_by() statement 

In [ ]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(cookies.c.quantity)
rp = engine.execute(s)
for cookie in rp:
    print(f'{cookie.quantity} - {cookie.cookie_name}')

We kunnen de order_by() ook direct toevoegen achter de select()

In [ ]:
s = select([cookies.c.cookie_name, cookies.c.quantity]).order_by(cookies.c.quantity)
rp = engine.execute(s)
for cookie in rp:
    print(f'{cookie.quantity} - {cookie.cookie_name}')

sort in reverse or descending order, use the desc() statement. 

In [ ]:
from sqlalchemy import desc
s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(desc(cookies.c.quantity))

# Limiting

the limit() function 

In [ ]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(cookies.c.quantity)
s = s.limit(2)
rp = engine.execute(s)
print([result.cookie_name for result in rp])

# Built-In SQL Functions and Labels

database functions SUM() and COUNT()



In [ ]:
from sqlalchemy.sql import func
s = select([func.sum(cookies.c.quantity)])
rp = engine.execute(s)
print(rp.scalar())

In [ ]:
s = select([func.count(cookies.c.cookie_name)])
rp = engine.execute(s)
record = rp.first()
print(record.keys())
print(record.count_1)

This column name is annoying and cumbersome.

, so the fourth count() function would be count_4. 

 fix this
via the label() function. 

In [ ]:
s = select([func.count(cookies.c.cookie_name).label('inventory_count')])
rp = engine.execute(s)
record = rp.first()
print(record.keys())
print(record.inventory_count)

# Filtering

Filtering queries is done by adding where() statements just like in SQL. 

A typical
where() clause has a column, an operator, and a value or column.

 It is possible to
chain multiple where() clauses together, and they will act like ANDs in traditional
SQL statements. 

In [ ]:
s = select([cookies]).where(cookies.c.cookie_name == 'chocolate chip')
rp = engine.execute(s)
record = rp.first()
print(record.items())
# print(record._mapping.items())

also use a where() statement to find all the cookie names that contain the
word “chocolate

In [ ]:
s = select([cookies]).where(cookies.c.cookie_name.like('%chocolate%'))
rp = engine.execute(s)
for record in rp.fetchall():
    print(record.cookie_name)

In the where() statement, we are using the cookies.c.cookie_name
column inside of a where() statement as a type of ClauseElement to filter our results.

# ClauseElements

ClauseElements are just an entity we use in a clause, 

and they are typically columns
in a table; 

however, unlike columns, ClauseElements come with many additional
capabilities.

![](images/clause_element_methods_sqlalchemy.png)

also the negation:
* notlike()
* notin_()

only isnot() does not follow the not naming convention (in front)

# Operators

all the standard comparison operators 
* == 
* !=
* <
* \>
* <=
* \>=

The ==
operator 
when compared to None, 
converts it
to an IS NULL statement. 

Arithmetic operators:
* \+
* \-
* \*
* /
* %

String concatenation with \+ : 

In [ ]:
s = select([cookies.c.cookie_name, 'SKU-' + cookies.c.cookie_sku])
for row in connection.execute(s):
    print(row)

compute values from multiple columns:

In [ ]:
from sqlalchemy import cast

s = select([cookies.c.cookie_name,
        cast((cookies.c.quantity * cookies.c.unit_cost), Numeric(12,2)).label('inv_cost')])
for row in engine.execute(s):
    print('{} - {}'.format(row.cookie_name, row.inv_cost))

Cast()  allows us to convert types.
 
label() function to rename the column. 

possible to accomplish the same task in Python with

In [ ]:
print('{} - {:.2f}'.format(row.cookie_name, row.inv_cost))

# Boolean Operators

SQL Boolean operators AND, OR, and NOT

=> bitwise logical operators (&, |, and ~)



### Python operator precedence rules:

when you write A < B & C < D,                               
what you are actually writing is A < (B&C) < D

# Conjunctions

 possible to chain multiple where() clauses together

 better 

 to use conjunctions


conjunctions in SQLAlchemy are:
* and_()
* or_()
* not_()


The and_() function:

In [ ]:
from sqlalchemy import and_, or_, not_
s = select([cookies]).where(
    and_(
        cookies.c.quantity > 23,
        cookies.c.unit_cost < 0.40
    )
)

for row in engine.execute(s):
    print(row.cookie_name)

The or_() function:

In [ ]:
from sqlalchemy import and_, or_, not_
s = select([cookies]).where(
    or_(
        cookies.c.quantity.between(10, 50),
        cookies.c.cookie_name.contains('chip')
    )
)
for row in engine.execute(s):
    print(row.cookie_name)

The not_() function:

In [ ]:
from sqlalchemy import and_, or_, not_
s = select([cookies]).where(
    not_(and_(
            cookies.c.quantity > 23,
            cookies.c.unit_cost < 0.40
        )
    )
)

for row in engine.execute(s):
    print(row.cookie_name)

# Updating Data

an update method with syn‐
tax almost identical to inserts, except that it can specify a where clause that indicates
which rows to update.


either the update() function or the update() method

In [ ]:
from sqlalchemy import update
u = update(cookies).where(cookies.c.cookie_name == "chocolate chip")
u = u.values(quantity=(cookies.c.quantity + 120))
result = engine.execute(u)

print(result.rowcount)

In [ ]:
s = select([cookies]).where(cookies.c.cookie_name == "chocolate chip")
result = engine.execute(s).first()

for key in result.keys():
    print('{:>20}: {}'.format(key, result[key]))

# Delete

the delete() function or the
delete() method

delete() takes no values parameter, only an optional where clause

In [ ]:
from sqlalchemy import delete
u = delete(cookies).where(cookies.c.cookie_name == "dark chocolate chip")
result = engine.execute(u)
print(result.rowcount)

In [ ]:
s = select([cookies]).where(cookies.c.cookie_name == "dark chocolate chip")
result = engine.execute(s).fetchall()
print(len(result))

OK, at this point, let’s load up some data using what we already learned for the users,
orders, and line_items tables. 

In [ ]:
customer_list = [
        {
        'username': 'cookiemon',
        'email_address': 'mon@cookie.com',
        'phone': '111-111-1111',
        'password': 'password'
        },
        {
        'username': 'cakeeater',
        'email_address': 'cakeeater@cake.com',
        'phone': '222-222-2222',
        'password': 'password'
        },
        {
        'username': 'pieguy',
        'email_address': 'guy@pie.com',
        'phone': '333-333-3333',
        'password': 'password'
        }
]

ins = users.insert()
result = engine.execute(ins, customer_list)

Now that we have customers, we can start to enter their orders and line items

In [ ]:
ins = insert(orders).values(user_id=1, order_id=1)
result = engine.execute(ins)

In [ ]:
ins = insert(line_items)
line_items_input = [
        {
        'order_id': 1,
        'cookie_id': 1,
        'quantity': 2,
        'extended_cost': 1.00
        },
        {
        'order_id': 1,
        'cookie_id': 3,
        'quantity': 12,
        'extended_cost': 3.00
        }
]
result = engine.execute(ins, line_items_input)

In [ ]:

ins = insert(orders).values(user_id=2, order_id=2)
result = engine.execute(ins)


In [ ]:
ins = insert(line_items)
line_items_input = [
        {
        'order_id': 2,
        'cookie_id': 1,
        'quantity': 24,
        'extended_cost': 12.00
        },
        {
        'order_id': 2,
        'cookie_id': 4,
        'quantity': 6,
        'extended_cost': 6.00
        }
]
result = engine.execute(ins, line_items_input)

# JOINS

## join() methods

With select_from(), we can replace the entire from
clause that SQLAlchemy would generate with one we specify

create select:

In [ ]:
columns = [
    orders.c.order_id, 
    users.c.username, 
    users.c.phone, 
    cookies.c.cookie_name, 
    line_items.c.quantity, 
    line_items.c.extended_cost
    ]

cookiemon_orders = select(columns)


Check output

In [ ]:
results = engine.execute(cookiemon_orders)
for row in results.fetchall():
    print(row)

join tables + check SQL

In [ ]:

cookiemon_orders = cookiemon_orders.select_from(orders.join(users).join(
line_items).join(cookies)).where(users.c.username ==
'cookiemon')

print(cookiemon_orders)    


execute SQL

In [ ]:
result = engine.execute(cookiemon_orders).fetchall()
for row in result:
    print(row)

## outerjoin() 

requires a bit more care in the ordering of the join, as the table we use the outer
join() method on will be the one from which all results are returned

create sql statement + check statement

In [ ]:
from sqlalchemy.sql import func

columns = [users.c.username, func.count(orders.c.order_id)]
all_orders = select(columns)
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)

print(all_orders)

execute sql

In [ ]:
result = engine.execute(all_orders).fetchall()
for row in result:
    print(row)

# Aliases

In [ ]:
employee_table = Table('employee', metadata,
    Column('id', Integer, primary_key=True),
    Column('manager', None, ForeignKey('employee.id')),
    Column('name', String(255))
)

suppose we want to select all the employees managed by an employee named
Fred

In SQL :

In [ ]:
SELECT employee.name
FROM employee, employee AS manager
WHERE employee.manager_id = manager.id
AND manager.name = 'Fred'

create query in alchemy

In [ ]:
manager = employee_table.alias('mgr')
stmt = select([employee_table.c.name],
    and_(
        employee_table.c.manager_id==manager.c.id,
        manager.c.name=='Fred'
        )
    )
print(stmt)

get results

In [ ]:
rp = engine.execute(stmt)
print(rp.fetchall())

SQLAlchemy can also choose the alias name automatically, 

In [ ]:
manager = employee_table.alias()
stmt = select([employee_table.c.name],
and_(employee_table.c.manager_id==manager.c.id,
manager.c.name=='Fred'))
print(stmt)

In [ ]:
rp = engine.execute(stmt)
print(rp.fetchall())

# Grouping

When using grouping, you need one or more columns to group on and one or more
columns that it makes sense to aggregate with counts, sums, etc., as you would in
normal SQL. 

In [ ]:
columns = [users.c.username, func.count(orders.c.order_id)]
all_orders = select(columns)
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)
result = engine.execute(all_orders).fetchall()
for row in result:
    print(row)

# Chaining

a function that got a list of
orders for us it might look like :

In [ ]:
def get_orders_by_customer(cust_name):
    columns = [orders.c.order_id, users.c.username, users.c.phone, cookies.c.cookie_name, line_items.c.quantity, line_items.c.extended_cost]
    cust_orders = select(columns)

    cust_orders = cust_orders.select_from(
        users.join(orders).join(line_items).join(cookies)
        )

    cust_orders = cust_orders.where(users.c.username == cust_name)
    result = engine.execute(cust_orders).fetchall()
    return result
    
get_orders_by_customer('cakeeater')

what if we wanted to get only the orders that have shipped

In [ ]:
def get_shipped_orders_by_customer(cust_name, shipped=None, details=False):
    columns = [orders.c.order_id, users.c.username, users.c.phone]
    joins = users.join(orders)

    if details:
        columns.extend([cookies.c.cookie_name, line_items.c.quantity, line_items.c.extended_cost])
        joins = joins.join(line_items).join(cookies)

    cust_orders = select(columns)
    cust_orders = cust_orders.select_from(joins)
    cust_orders = cust_orders.where(users.c.username == cust_name)

    if shipped is not None:
        cust_orders = cust_orders.where(orders.c.shipped == shipped)
    
    result = engine.execute(cust_orders).fetchall()
    return result

In [ ]:
get_shipped_orders_by_customer('cakeeater')

In [ ]:
get_shipped_orders_by_customer('cakeeater', details=True)

In [ ]:
get_shipped_orders_by_customer('cakeeater', shipped=True)

In [ ]:
get_shipped_orders_by_customer('cakeeater', shipped=False)

In [ ]:
get_shipped_orders_by_customer('cakeeater', shipped=False, details=True)

# Raw Queries

execute raw SQL statements 

In [ ]:
result = engine.execute("select * from orders").fetchall()
print(result)

 I will often use small text snippets to help
make a query clearer

In [ ]:
from sqlalchemy import text
stmt = select([users]).where(text("username='cookiemon'"))
print(stmt)

In [ ]:
print(engine.execute(stmt).fetchall())

What did it look like without "text"

In [ ]:
stmt = select([users]).where(users.c.username == 'cookiemon')
print(stmt)